# TP2 - EA3
Diseñar, simular e implemetar un amplificador de microondas de baja señal para máxima transferencia de energía (adaptación de impedancias de entrada y salida)
Los requisitos son:
- $Z_{in}$ = 50 $\Omega$
- $Z_L$ = 50 $\Omega$
- FR4 $\rightarrow \epsilon_r = 4.5$
- El transistor elegido es BFP 450
 A continuacion se cargan en un arreglo los parámetros S en base a la tensión y corriente de polarización seleccionadas para el transistor

### Se procede a cargar los datos del archivo s2p para poder trabajar con ellos

In [1]:
import numpy as np
import skrf as rf # se hace uso de la libreria scikit-rf para leer los datos brindados por el fabricante
file_path = 'E:\Materias en curso\Analogica 3\TP2\SPAR\BFP450\BFP450_w_noise_VCE_1.0V_IC_0.16A.s2p'
network = rf.Network(file_path)
print("Información de la red:",network)
# Acceder a los parámetros S
s_parameters = network.s

Información de la red: 2-Port Network: 'BFP450_w_noise_VCE_1.0V_IC_0.16A',  1.2-12.0 GHz, 154 pts, z0=[50.+0.j 50.+0.j]


### Parámetros S

In [2]:
S11=s_parameters[0, 0][0]
S12=s_parameters[0, 0][1]
S21=s_parameters[0, 1][0]
S22=s_parameters[0, 1][1]
print(f"Coeficiente S11:", "{:.4f}".format(S11))
print(f"Coeficiente S12:", "{:.4f}".format(S12))
print(f"Coeficiente S21:", "{:.4f}".format(S21))
print(f"Coeficiente S22:", "{:.4f}".format(S22))

Coeficiente S11: -0.7819+0.2496j
Coeficiente S12: 0.0249+0.0443j
Coeficiente S21: 1.3161+3.3411j
Coeficiente S22: -0.6122+0.1245j


### Matriz de coeficientes y su Determinante

In [3]:
Mat_S = [[S11, S12], [S21 , S22]]
# Redondear los valores de la matriz para la impresión
Mat_S_rounded = [[np.round(S, 4) for S in row] for row in Mat_S]
# Imprimir la matriz Mat_S con valores redondeados a 4 decimales
print("Matriz de Coeficientes S:")
for row in Mat_S_rounded:
    row_str = " ".join(["{:.4f}".format(val) for val in row])
    print(row_str)

print("")
det = np.linalg.det(Mat_S)
print("Su determinante es:", "{:.4f}".format(det))

Matriz de Coeficientes S:
-0.7819+0.2496j 0.0249+0.0443j
1.3161+3.3411j -0.6122+0.1245j

Su determinante es: 0.5626-0.3917j


### Coeficiente de Rollet 

In [4]:
K=(1+((abs(det))**2)-((abs(S22))**2)-((abs(S11))**2))/abs(2*S12*S21)
print("El valor de K es:","{:.4f}".format(K))

El valor de K es: 1.1129


### Se procede a calcular los coeficientes B1, B2, C1 y C2

In [5]:
B1=1+((abs(S11))**2)-((abs(S22))**2)-(abs(det))**2
B2=1+((abs(S22))**2)-((abs(S11))**2)-(abs(det))**2
C1=S11-(det*np.conj(S22))
C2=S22-(det*np.conj(S11))
print("B1=","{:.4f}".format(abs(B1)), "{:.4f}".format(np.degrees(np.angle(B1))),"°")
print("B2=","{:.4f}".format(abs(B2)), "{:.4f}".format(np.degrees(np.angle(B2))),"°")
print("C1=","{:.4f}".format(abs(C1)), "{:.4f}".format(np.degrees(np.angle(C1))),"°")
print("C2=","{:.4f}".format(abs(C2)), "{:.4f}".format(np.degrees(np.angle(C2))),"°")

B1= 0.8134 0.0000 °
B2= 0.2465 0.0000 °
C1= 0.3968 168.3951 °
C2= 0.0852 -150.9512 °


### Coeficiente de reflexión de entrada Γ<sub>in</sub>

In [6]:
gamma_in=(B1-np.sqrt((B1**2)-(4*(abs(C1)**2))))/(2*abs(C1))
print("Γᵢₙ: ","{:.4f}".format(gamma_in), "{:.4f}".format(np.degrees(np.angle(C1))),"°")
gamma_in_rect=gamma_in * np.cos(np.angle(C1)) + gamma_in * np.sin(np.angle(C1)) * 1j
print("Γᵢₙ_rect","{:.4f}".format(gamma_in_rect))

Γᵢₙ:  0.8003 168.3951 °
Γᵢₙ_rect -0.7840+0.1610j


### Coeficiente de reflexión de salida Γ<sub>out</sub>

In [9]:
gamma_out=(B2-np.sqrt((B2**2)-(4*(abs(C2)**2))))/(2*abs(C2))
print("Γₒᵤₜ ","{:.4f}".format(gamma_out), "{:.4f}".format(np.degrees(np.angle(C2))),"°")
gamma_out_rect=gamma_out * np.cos(np.angle(C2)) + gamma_out * np.sin(np.angle(C2)) * 1j
print("Γₒᵤₜ_rect","{:.4f}".format(gamma_out_rect))

Γₒᵤₜ  0.4010 -150.9512 °
Γₒᵤₜ_rect -0.3506-0.1947j


### Z<sub>in</sub> y Z<sub>out</sub>:

In [10]:
Zo=50
Z_in=Zo*((1+gamma_in_rect)/(1-gamma_in_rect))
print("Z_in=","{:.4f}".format(Z_in))
Z_out=Zo*((1+gamma_out_rect)/(1-gamma_out_rect))
print("Z_out=","{:.4f}".format(Z_out))

Z_in= 5.6014+5.0179j
Z_out= 22.5361-10.4573j
